In [8]:
# Import dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import requests
import time
from datetime import datetime
# Import the API key.
from config import weather_api_key

In [13]:
### Create a new set of 2,000 random latitudes and longitudes.
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

### Get the nearest city using the citipy module.
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

### Perform an API call with the OpenWeatherMap.
# Create an empty list to hold the weather data.
city_data = []

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data. ### Retrieve the following information from the API call:
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"] # Latitude and longitude
        city_lng = city_weather["coord"]["lon"] # Latitude and longitude
        city_max_temp = city_weather["main"]["temp_max"] # Maximum temperature
        city_humidity = city_weather["main"]["humidity"] # Percent humidity
        city_clouds = city_weather["clouds"]["all"] # Percent cloudiness
        city_wind = city_weather["wind"]["speed"] # Wind speed 
        city_descr = city_weather['weather'][0]['description'] # Weather description (for example, clouds, fog, light rain, clear sky)

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country":city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Weather Description": city_descr})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")



Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | passy
Processing Record 3 of Set 1 | carnarvon
Processing Record 4 of Set 1 | torbay
Processing Record 5 of Set 1 | namie
Processing Record 6 of Set 1 | mataura
Processing Record 7 of Set 1 | saint-francois
Processing Record 8 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 9 of Set 1 | limon
Processing Record 10 of Set 1 | bathsheba
Processing Record 11 of Set 1 | praia da vitoria
Processing Record 12 of Set 1 | broken hill
Processing Record 13 of Set 1 | hobart
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | severo-kurilsk
Processing Record 16 of Set 1 | cabras
Processing Record 17 of Set 1 | chokwe
Processing Record 18 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 19 of Set 1 | hasaki
Processing Record 20 of Set 1 | puerto ayora
Processing Record 21 of Set 1 | carutapera
Processing Record

In [15]:
### Add the data to a new DataFrame.
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
new_column_order = ['City','Country','Lat','Lng','Max Temp','Humidity',
'Cloudiness','Wind Speed','Weather Description']

city_data_df = city_data_df[new_column_order]
city_data_df.head(5)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,Ushuaia,AR,-54.8000,-68.3000,38.86,81,75,5.75,broken clouds
1,Passy,FR,45.9203,6.6877,68.49,71,65,3.80,broken clouds
2,Carnarvon,AU,-24.8667,113.6333,71.02,70,100,9.98,overcast clouds
3,Torbay,CA,47.6666,-52.7314,55.27,93,100,9.22,overcast clouds
4,Namie,JP,37.4833,141.0000,96.71,82,64,4.32,broken clouds


In [16]:
# Export the DataFrame as WeatherPy_Database.csv into the Weather_Database folder
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")